# Stack Neuron

### Import Data

In [51]:
import pandas as pd
import numpy as np
data = pd.read_csv("iris.csv")
data = data[:100]
N = data.shape[0]

list_species=[]
for i in range(len(data.index)):
    x = data["species"][i]
    if x not in list_species:
        list_species.append(x)
data["species"] =data["species"].replace(to_replace=list_species, value=range(len(list_species)))

X = data.drop("species", 1)
y = data["species"]

# Logistic Regresion

#### initialize w and b

In [52]:
w_logistic, b_logistic = np.zeros(4), 0

#### sigmoid

In [53]:
def sigmoid(a):
    return 1.0 / (1.0 + np.exp(-a))

#### gradient 

In [54]:
def grad_logistic(x, w, b):
    sig = sigmoid(np.dot(x, w) + b)
    w_gradient = np.multiply(x, ((1-sig))[:, np.newaxis])
    b_gradient = ((1-sig))
    return w_gradient, b_gradient

#### cross entropy and grad

In [55]:
def cross_entropy_grad(x, w, b, y):
    w_gradient, b_gradient = grad_logistic(x, w, b)
    c_w_gradient = np.matmul(np.transpose(sigmoid(np.dot(x, w) + b)-y), x) /N
    c_b_gradient = sum(np.transpose(sigmoid(np.dot(x, w) + b)-y)) /N
    return c_w_gradient, c_b_gradient

def cross_entropy(y_pred, y):
    cross_entropy = -(1/N) * sum ((y * np.log(y_pred)) + (1-y) * np.log(1 - y_pred) )
    return cross_entropy

#### Stochastic Gradient Descent

In [56]:
def sgd_logistic(x, initial_w, initial_b, lowtrain, uptrain, iterations):
    place_w = initial_w
    place_b = initial_b
    for i in range(iterations):
        trainrate=np.random.uniform(lowtrain,uptrain,1)
        c_w_place, c_b_place = cross_entropy_grad(x,place_w,place_b, y)
        place_w = place_w - trainrate * c_w_place
        place_b = place_b - trainrate * c_b_place
    y_pred = sigmoid(np.dot(x, place_w) + place_b)
    ce = cross_entropy(y_pred, y)
    return place_w, place_b, ce, y_pred

### check the accuracy

#### shuffling instances

In [57]:
indexer = np.arange(X.shape[0])
np.random.shuffle(indexer)
X = X.iloc[indexer, ]
y = y.iloc[indexer, ]

In [61]:
from sklearn.metrics import mean_squared_error
y_pred = sgd_logistic(X, w_logistic, b_logistic, 0.01, 0.01, 10000)[3]
y_true = np.array(y)
rmse_original_logistic = mean_squared_error(y_true, y_pred)
print ("rmse of original logistic regression is")
print (rmse_original_logistic)

rmse of original logistic regression is
0.000162506167431


In [62]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(X, y)
y_pred_sckit_logistic = clf.predict_proba(X)[:,1]
rmse_sckit_logistic = mean_squared_error(y_true, y_pred_sckit_logistic)
print ("rmse of sckit logistic regression is")
print (rmse_sckit_logistic)

rmse of sckit logistic regression is
0.00104559004721


# Polynomial Regression

#### initialize w and b

In [42]:
w1_, w2, b = np.zeros(4), np.zeros(4), 0
x2 = np.multiply(X, X)

#### gradient

In [43]:
def grad_polynomial(x, w1, w2, b, y):
    w1_gradient = np.matmul(np.transpose(np.dot(x, w1) + b + np.dot(x2, w2) -y), x) /N
    w2_gradient = np.matmul(np.transpose(np.dot(x, w1) + b + np.dot(x2, w2) -y), x2) /N
    b_gradient = sum(np.transpose(np.dot(x, w1) + b + np.dot(x2, w2) - y)) /N
    return w1_gradient, w2_gradient, b_gradient

In [44]:
grad_polynomial(X, w1, w2, b, y)

(array([-2.968, -1.385, -2.13 , -0.663]),
 array([-17.7486,  -3.8847,  -9.182 ,  -0.8983]),
 -0.5)

#### Stochastic Gradient

In [45]:
def sgd_poly(x, initial_w1, initial_w2, initial_b, lowtrain, uptrain, iterations):
    place_w1 = initial_w1
    place_w2 = initial_w2
    place_b = initial_b
    for i in range(iterations):
        trainrate=np.random.uniform(lowtrain,uptrain,1)
        c_w1_place, c_w2_place, c_b_place = grad_polynomial(x, place_w1, place_w2, place_b, y)
        place_w1 = place_w1 - trainrate * c_w1_place
        place_w2 = place_w2 - trainrate * c_w2_place
        place_b = place_b - trainrate * c_b_place
    y_pred = np.dot(x, place_w1) + np.dot(x2, place_w2) + place_b
    return place_w1, place_w2, place_b, y_pred

### check the accuracy

#### shuffling instances

In [46]:
indexer = np.arange(X.shape[0])
np.random.shuffle(indexer)
X = X.iloc[indexer, ]
y = y.iloc[indexer, ]

In [47]:
from sklearn.metrics import mean_squared_error
y_pred = sgd_poly(X, w1, w2, b, 0.000001, 0.0001,10000)[3]
y_true = np.array(y)
rmse_original_logistic = mean_squared_error(y_true, y_pred)
print ("rmse of original polynomial regression is")
print (rmse_original_logistic)

rmse of original polynomial regression is
0.0315617589377


# Stacking 